YOLO11 Model Training Script
===========================
Train YOLO11n model on CARLA-generated dataset (single-map specialization).

Usage examples:

    # Full-model fine-tuning (recommended for single CARLA map)
    python train_yolo.py \
        --dataset yolo_carla_dataset \
        --model yolo11n.pt \
        --epochs 250 \
        --batch 16 \
        --augment \
        --pretrained

    # Experiment: freeze first 10 layers (more generic backbone)
    python train_yolo.py \
        --dataset yolo_carla_dataset \
        --model yolo11n.pt \
        --epochs 120 \
        --batch 16 \
        --augment \
        --pretrained \
        --freeze 10

In [ ]:
!pip install ultralytics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.6 MB/s eta 0:00:00


In [ ]:
import os
import shutil
import argparse

from pathlib import Path
from types import SimpleNamespace

import torch
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
def check_gpu():
    """Check GPU availability"""
    if torch.cuda.is_available():
        print(f"✓ GPU available: {torch.cuda.get_device_name(0)}")
        print(f"  CUDA version: {torch.version.cuda}")
        print(f"  GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
        return True
    else:
        print("⚠ No GPU available. Training will use CPU (slower)")
        return False

In [ ]:
def train_model(args):
    """Train YOLO model"""
    print("\n=== YOLO Training Configuration ===")
    print(f"Dataset dir : {args.dataset}")
    print(f"Model       : {args.model}")
    print(f"Epochs      : {args.epochs}")
    print(f"Batch size  : {args.batch}")
    print(f"Image size  : {args.imgsz}")
    print(f"Workers     : {args.workers}")
    print(f"Optimizer   : {args.optimizer}")
    print(f"LR0         : {args.lr0}")
    print(f"Patience    : {args.patience}")
    print(f"Freeze      : {args.freeze}  (0 = train full model)")
    print(f"Project     : {args.project}")
    print(f"Run name    : {args.name}")
    print(f"Pretrained  : {args.pretrained}")
    print(f"Augment     : {args.augment}")
    print(f"Cache       : {args.cache}")
    print(f"Resume      : {args.resume}")
    print(f"Validate    : {args.validate}")

    # Check dataset.yaml exists
    yaml_path = os.path.join(args.dataset, "dataset.yaml")
    if not os.path.exists(yaml_path):
        print(f"\n✗ Error: dataset.yaml not found at {yaml_path}")
        return

    # Load model
    print(f"\nLoading model: {args.model}")
    model = YOLO(args.model)

    # Check GPU
    has_gpu = check_gpu()
    device = 0 if has_gpu and not args.cpu else "cpu"
    print(f"\nUsing device: {device}")

    # Start training
    print("\n=== Starting Training ===\n")

    results = model.train(
        data=yaml_path,
        epochs=args.epochs,
        imgsz=args.imgsz,
        batch=args.batch,
        workers=args.workers,
        device=device,
        project=args.project,
        name=args.name,
        pretrained=args.pretrained,
        optimizer=args.optimizer,
        lr0=args.lr0,
        patience=args.patience,
        save=True,
        save_period=args.save_period,
        cache=args.cache,
        augment=args.augment,
        resume=args.resume,
        verbose=True,
        freeze=args.freeze,   # <-- NEW: control how much of the backbone is frozen
    )

    print("\n=== Training Complete ===")
    print(f"Results saved to: {results.save_dir}")

    # ---- NEW: save final model somewhere explicit ----
    best_weights = Path(results.save_dir) / "weights" / "best.pt"
    last_weights = Path(results.save_dir) / "weights" / "last.pt"

    print(f"Best weights: {best_weights}")
    print(f"Last weights: {last_weights}")

    if args.final_model_path:
        final_path = Path(args.final_model_path)
        final_path.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy2(best_weights, final_path)
        print(f"✔ Copied best weights to: {final_path}")

    # Optional validation
    if args.validate:
        print("\n=== Running Validation ===")
        metrics = model.val()
        print("\nValidation Results:")
        print(f"  mAP50    : {metrics.box.map50:.4f}")
        print(f"  mAP50-95 : {metrics.box.map:.4f}")
        print(f"  Precision: {metrics.box.mp:.4f}")
        print(f"  Recall   : {metrics.box.mr:.4f}")

    return results

In [ ]:
def main():
    parser = argparse.ArgumentParser(description="Train YOLO11 model on CARLA dataset")

    # Dataset
    parser.add_argument(
        "--dataset",
        type=str,
        required=True,
        help="Path to dataset directory containing dataset.yaml",
    )

    # Model
    parser.add_argument(
        "--model",
        type=str,
        default="yolo11n.pt",
        help="Model to use (yolo11n/s/m/l/x.pt)",
    )

    # Training parameters
    parser.add_argument(
        "--epochs",
        type=int,
        default=250,  # higher by default – okay to overfit to single map
        help="Number of training epochs",
    )
    parser.add_argument(
        "--batch",
        type=int,
        default=16,
        help="Batch size (-1 for auto)",
    )
    parser.add_argument(
        "--imgsz",
        type=int,
        default=640,
        help="Image size for training",
    )
    parser.add_argument(
        "--workers",
        type=int,
        default=8,
        help="Number of dataloader workers",
    )

    # Optimization
    parser.add_argument(
        "--optimizer",
        type=str,
        default="AdamW",
        choices=["SGD", "Adam", "AdamW", "auto"],
        help="Optimizer",
    )
    parser.add_argument(
        "--lr0",
        type=float,
        default=0.005,
        help="Initial learning rate",
    )
    parser.add_argument(
        "--patience",
        type=int,
        default=60,
        help="Epochs to wait for no improvement for early stopping",
    )

    # Save options
    parser.add_argument(
        "--project",
        type=str,
        default="runs/train",
        help="Project directory",
    )
    parser.add_argument(
        "--name",
        type=str,
        default="carla_yolo11n_one_map",
        help="Experiment name",
    )
    parser.add_argument(
        "--save-period",
        type=int,
        default=-1,
        help="Save checkpoint every x epochs (-1 for disabled)",
    )

    # Advanced / behavior options
    parser.add_argument(
        "--pretrained",
        action="store_true",
        help="Use pretrained weights (recommended)",
    )
    parser.add_argument(
        "--cache",
        action="store_true",
        help="Cache images for faster training",
    )
    parser.add_argument(
        "--augment",
        action="store_true",
        help="Apply data augmentation",
    )
    parser.add_argument(
        "--cpu",
        action="store_true",
        help="Force CPU usage",
    )
    parser.add_argument(
        "--resume",
        action="store_true",
        help="Resume training from last checkpoint",
    )
    parser.add_argument(
        "--validate",
        action="store_true",
        help="Run validation after training",
    )

    # NEW: freeze control
    parser.add_argument(
        "--freeze",
        type=int,
        default=0,
        help="Number of backbone layers to freeze (0 = train full model)",
    )

    parser.add_argument(
        "--final-model-path",
        type=str,
        default="",
        help="Optional path to copy best.pt after training (e.g. inside Drive)",
    )

    args = parser.parse_args()

    # Train
    train_model(args)


In [ ]:
# from train_yolo import train_model  # <-- delete this line

dataset_root = "/content/drive/MyDrive/datasets/cmpe789/yolo_carla_dataset"

args = SimpleNamespace(
    dataset=dataset_root,
    model="yolo11n.pt",
    epochs=250,
    batch=128,
    imgsz=640,
    workers=8,
    optimizer="AdamW",
    lr0=0.005,
    patience=60,
    project="/content/drive/MyDrive/datasets/cmpe789/cmpe789_yolo_runs",
    name="carla_yolo11n_one_map",
    save_period=-1,
    pretrained=True,
    cache=True,
    augment=True,
    cpu=False,
    resume=False,
    validate=True,
    freeze=0,
    final_model_path="/content/drive/MyDrive/datasets/cmpe789/cmpe789_yolo_runs/yolo11n_merged.pt",
)

results = train_model(args)


=== YOLO Training Configuration ===
Dataset dir : /content/drive/MyDrive/datasets/cmpe789/yolo_carla_dataset
Model       : yolo11n.pt
Epochs      : 250
Batch size  : 128
Image size  : 640
Workers     : 8
Optimizer   : AdamW
LR0         : 0.005
Patience    : 60
Freeze      : 0  (0 = train full model)
Project     : /content/drive/MyDrive/datasets/cmpe789/cmpe789_yolo_runs
Run name    : carla_yolo11n_one_map
Pretrained  : True
Augment     : True
Cache       : True
Resume      : False
Validate    : True

Loading model: yolo11n.pt
✓ GPU available: NVIDIA L4
  CUDA version: 12.6
  GPU memory: 23.80 GB

Using device: 0

=== Starting Training ===

Ultralytics 8.3.237 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=128, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, 